In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

#pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
data_proc = pd.read_csv("../../../data/Data_processed.csv", low_memory=False)
data_cyt = pd.read_csv("../../../data/Cytokines_3Dec2019.csv", low_memory=False)

# Patients in ICU per day

In [4]:
for i in range(1,8):
    tmp = data_proc.loc[data_proc["day"] == i,].shape[0]
    print("Day {0}: {1}".format(i, tmp))

print("Total patients: {0}".format(data_proc.shape[0]))

Day 1: 17
Day 2: 27
Day 3: 51
Day 4: 57
Day 5: 52
Day 6: 19
Day 7: 186
Total patients: 409


# Overlap cytokine dataset - big dataset

## Prepare cytokine dataset

In [5]:

print("\nPatients cytokine dataset (original): {}".format(data_cyt.shape[0]))

# Remove unannotated patients
data_cyt = data_cyt[-data_cyt["PatientID"].isna()]
data_cyt.loc[:,"PatientID"] = data_cyt["PatientID"].astype(str).str[:-2].astype("int64")
print("Patients cytokine dataset (after removal unannotated patients): {}".format(data_cyt.shape[0]))

# Take only cytokine information
data_cytOnly = pd.concat([data_cyt.loc[:, ["PatientID", "day"]], data_cyt.iloc[:, 27:]], axis = 1)

# New variables for cytokine day 0 / day 3
cyt_day0 = data_cytOnly[data_cytOnly["day"] == 0]
cyt_day3 = data_cytOnly[data_cytOnly["day"] == 3]

cyt_merge = pd.merge(cyt_day0, cyt_day3, on=["PatientID"], how = "outer", suffixes=['_1', '_3'])
cyt_merge.drop(["day_1", "day_3"], axis=1, inplace=True)

print("Patients cytokine dataset (merging duplicates): {}\n".format(cyt_merge.shape[0]))



Patients cytokine dataset (original): 400
Patients cytokine dataset (after removal unannotated patients): 381
Patients cytokine dataset (merging duplicates): 291



In [6]:
data_cytOnly["day"].value_counts()

day
0    291
3     90
Name: count, dtype: int64

## Merge cytokine and big dataset

In [7]:
# Merge with other dataset via PatientID
bigCyt_leftmerge = pd.merge(data_proc, cyt_merge, left_on='PATIENT_ID', right_on='PatientID', how = "left")
bigCyt_innermerge = pd.merge(data_proc, cyt_merge, left_on='PATIENT_ID', right_on='PatientID', how = "inner")

In [10]:
bigCyt_leftmerge.to_csv("../../../results/datasets/allFeatures_fullDataset.csv", index=False)
bigCyt_innermerge.to_csv("../../../results/datasets/allFeatures_overlapDataset.csv", index=False)

In [12]:
bigCyt_leftmerge.shape

(409, 2488)